In [7]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import platform

In [8]:
class CovidTimeSeries(object):
    """ Stores the JHU time series data for a county for covid """
    def __init__(self):
        self.dates = None
        self.regionCode= None
        self.regionName=None
        self.positive=None  #Infected used by the paper
        self.Lat=None 
        self.Long=None
        self.Combined_Key=None #unused
        self.healed=None   #Recovered
        self.totalCases=None
        self.tested = None
        self.deaths=None 

class CovidDatabase(object):
    """ Stores the covid-19 data"""
    def __init__(self):
        self.CovidData={}
        self.DateRange=[]
      
    def loadTimeSeries(self, filenameI, startdate, enddate):
        """ load the infections data from filenameI and death data from filenameD
            from startdate to enddate
        """
        csvfile=open(filenameI, newline='',  encoding='UTF-8')
        rd = csv.reader(csvfile, delimiter=',')
        data=[]
        for lv in rd:
                data.append(lv)

        header=data[0]
        
        infectionData=(data[1:])
        temp = np.array(infectionData)
        dates = temp[:,0]
        dates = dates.tolist()


        startdate_index=dates.index(startdate)
        enddate_index=dates.index(enddate)
       

        CountyD={}
        N=len(infectionData);
        for i in range(N):
            if not startdate<= infectionData[i][0] <= enddate:
                continue
            if int(float(infectionData[i][2])) not in CountyD: #if key not already initialized
            
                c1=CovidTimeSeries()
                c1.regionCode = infectionData[i][2]
                c1.regionName = infectionData[i][3]
                c1.Lat = float(infectionData[i][4])
                c1.Long = float(infectionData[i][5])
                dRange = enddate_index - startdate_index + 1
                fp=infectionData[i][2]
                x=int(float(fp))
                c1.dates = [infectionData[i][0]]
                c1.deaths = [float(infectionData[i][14])]
                c1.positive = [float(infectionData[i][10])]
                c1.healed = [float(infectionData[i][13])]
                if(infectionData[i][19] == ''):
                    c1.tested = [float('0')]
                else:
                    c1.tested = [float(infectionData[i][19])]
                CountyD[x]=c1
            else: #if key already initialized
                fp=infectionData[i][2]
                x=int(float(fp))
                CountyD[x].positive.append(float(infectionData[i][10]))
                CountyD[x].deaths.append(float(infectionData[i][14]))
                CountyD[x].dates.append(infectionData[i][0])
                CountyD[x].healed.append(float(infectionData[i][13]))
                if(infectionData[i][19] == ''):
                    CountyD[x].tested.append(float('0'))
                else:
                    CountyD[x].tested.append(float(infectionData[i][19]))
        for key in CountyD: #Turn the lists into arrays
            CountyD[key].deaths = np.array(CountyD[key].deaths)
            CountyD[key].positive = np.array(CountyD[key].positive)
            CountyD[key].healed = np.array(CountyD[key].healed)
            CountyD[key].totalCases = np.array(CountyD[key].totalCases)
            CountyD[key].tested = np.array(CountyD[key].tested)
            CountyD[key].dates = np.array(CountyD[key].dates)
        self.DateRange=dates
        self.CovidData=CountyD
        

In [11]:
# Modify the path syntax based on the current platform
if platform.system() == "Windows":
    pathc="..\\Data\\Italian Data\\"
elif platform.system() == "Linux":
    pathc="../Data/Italian Data/"

fullDataName="dpc-covid19-ita-regioni.csv"

startdate='2020-02-24T18:00:00'
enddate='2021-05-18T17:00:00'

database=CovidDatabase();
database.loadTimeSeries(pathc+fullDataName, startdate, enddate)

In [10]:
CountyD=database.CovidData
province = ['Emilia-Romagna']
for key in CountyD:
    wanted = np.zeros((np.shape(CountyD[key].dates)[0],5))
    wanted = wanted.tolist()
    break
for i in province:
    for key in CountyD:
        if CountyD[key].regionName == i:
            print("test")
            tempDeaths = CountyD[key].deaths
            tempRecovered = CountyD[key].healed
            tempInfected = CountyD[key].positive
            tempTested = CountyD[key].tested
            tempDates = CountyD[key].dates
            tempRegionalName = CountyD[key].regionName
            tempRegionalCode = CountyD[key].regionCode
            for N in range(np.shape(CountyD[key].dates)[0]):
                wanted[N][0] = tempDates[N]
                wanted[N][1] = tempInfected[N] 
                wanted[N][2] = tempTested[N]
                wanted[N][3] = tempRecovered[N]
                wanted[N][4] = tempDeaths[N]
filename = "Emilia-Romanga.csv"
fields = ['Dates', 'Infected', 'Tested', 'Recovered', 'Deaths']
# writing to csv file 
with open(filename, 'w', newline = '') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile, delimiter = ',') 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(wanted)

test


# National Data

In [6]:
CountyD=database.CovidData
[N] = np.shape(CountyD[13].positive)
dates = CountyD[13].dates
print(N)

popITA = 60000000

infectRaw = [0] * N
recovRaw = [0] * N
deadRaw = [0] * N
natTested = [0] * N
wanted = np.zeros((np.shape(CountyD[13].dates)[0],5))
wanted = wanted.tolist()

for key in CountyD:
    natTested += CountyD[key].tested
    infectRaw += CountyD[key].positive
    recovRaw += CountyD[key].healed
    deadRaw += CountyD[key].deaths
    
for N in range(np.shape(natTested)[0]):
                wanted[N][0] = dates[N]
                wanted[N][1] = infectRaw[N] 
                wanted[N][2] = natTested[N]
                wanted[N][3] = recovRaw[N]
                wanted[N][4] = deadRaw[N]
                
filename = "National Data.csv"
fields = ['Dates', 'Infected', 'Tested', 'Recovered', 'Deaths']
# writing to csv file 
with open(filename, 'w', newline = '') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile, delimiter = ',') 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(wanted)

450
